In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import os
import gc
import skimage
import random

In [7]:
DATA_PATH = 'C:/Users/Nikita/Desktop/T-Systems/'

In [8]:
#X_MEAN = np.load('x_mean.npy')

In [12]:
def load_data(path, size_for_control):
    X = []
    y = np.concatenate([np.tile(row, (size_for_control, 1)) for row in np.eye(6)])
    for c in os.listdir(path):
        for image in os.listdir(path + '/' + c)[:size_for_control]:
            try:
                file_path = path + '/' + c + '/' + image
                img = Image.open(file_path).convert('L')
                img_data = np.asarray(img, dtype=np.int16)
                X.append(img_data)
            except:
                print('Error: ' + file_path)
    X = np.asarray(X)
    return X.reshape(len(X), *X[0].shape, 1), y

def shuffle_data(X, y):
    idx = list(range(len(X)))
    np.random.shuffle(idx)
    return X[idx], y[idx]

def preprocess(X):
    return (X.astype('float32')) / 255

In [13]:
train_size = 7500
X, y = load_data(DATA_PATH + 'TrainGray', train_size)

In [ ]:
X.shape, y.shape

In [2]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".png")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(d)
    for i in range(len(images)):
        images[i] = images[i]/255
    c = list(zip(images, labels))
    random.shuffle(c)
    images, labels = zip(*c)
    labels= [int(i) for i in labels]
    
    return np.array(images), np.array(labels)

ROOT_PATH = "C:/Users/Nikita/Desktop/T-Systems"
train_data_directory = os.path.join(ROOT_PATH, "TrainGray")
images, labels = load_data(train_data_directory)
images.shape, labels.shape

((45000, 100, 150), (45000,))

In [3]:
def get_data(t):
    
    images_test = images[t*9000:(t+1)*9000]
    labels_test = labels[t*9000:(t+1)*9000]
    
    #images_test = np.array(images_test)
    #labels_test = np.array(labels_test)
    
    images_train = np.concatenate((images[0:t*9000], images[(t+1)*9000:]))
    labels_train = np.concatenate((labels[0:t*9000], labels[(t+1)*9000:]))
    
    return images_train, labels_train, images_test, labels_test

class_names = ['button', 'checkbox', 'radiobutton', 'slider', 'spinner', 'textfield']

In [5]:
for i in range(5):
    model = keras.Sequential([
        keras.layers.Conv2D(input_shape=(100, 150, 1), kernel_size = (3, 3), filters = 32, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (3, 3), filters = 64, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (5, 5), filters = 128, activation='relu'),
        keras.layers.Conv2D(kernel_size = (3, 3), filters = 64, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (5, 5), filters = 64, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(6, activation='softmax')
    ])
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
              optimizer=tf.keras.optimizers.Adadelta(), 
              metrics=['accuracy'])
    
    images_train, labels_train, images_test, labels_test = get_data(i)
    
    images_train = images_train.reshape(-1, 100, 150, 1)
    images_test = images_test.reshape(-1, 100, 150, 1)
    
    model.fit(images_train, labels_train, epochs=30)
    
    loss, acc = model.evaluate(images_test, labels_test)
    print("Result: " + str(acc))
    
    del images_train
    del labels_train
    del images_test
    del labels_test
    
    gc.collect()

W0716 16:34:26.579456  6064 deprecation.py:323] From c:\users\nikita\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 36000 samples
Epoch 1/30
36000/36000 [==============================] - 35s 986us/sample - loss: 1.7906 - accuracy: 0.1767
Epoch 2/30
36000/36000 [==============================] - 32s 891us/sample - loss: 1.7883 - accuracy: 0.1842
Epoch 3/30
36000/36000 [==============================] - 32s 895us/sample - loss: 1.7847 - accuracy: 0.1781
Epoch 4/30
36000/36000 [==============================] - 33s 909us/sample - loss: 1.7792 - accuracy: 0.1745
Epoch 5/30
36000/36000 [==============================] - 33s 916us/sample - loss: 1.7719 - accuracy: 0.1695
Epoch 6/30
36000/36000 [==============================] - 34s 939us/sample - loss: 1.7652 - accuracy: 0.1690 29s - loss: 1 - ETA: 27s - loss: 1.7640 - accuracy: - ETA: 27s - loss: 1.7643 - accuracy: 0.17 - ETA: 27s - loss: 1.764 - ETA: 26s - loss: 1.7657 - accuracy: 0.1 - ETA: 25s - loss: 1.7656 - accuracy: 0.172 - ETA: 25s  - ETA: 23s - loss: 1.7657 - - ETA: 22s - loss: 1.7660 - accuracy: 0.1 - ETA: 22s - loss: 1.7661 - accura 

36000/36000 [==============================] - 34s 938us/sample - loss: 1.5903 - accuracy: 0.35423s - loss: 1.6169 - accuracy: 0.338 - ETA: 33s - loss: - ETA: 31s - loss: 1.6153 - accuracy - ETA: 31s - loss: 1.6131 - accuracy: 0 - ETA: 30s - loss: 1.6121 - accur - ETA: 30s - loss: 1.614 - ETA: 23s - loss: 1.6093 -  - ETA: 22s - loss: 1.6084 - accuracy: - ETA: 22s - loss: 1.6078  - ETA: 21s - loss: 1.6068 - accuracy: 0. - ETA: 20s - lo - ETA: 19s - loss: 1.6030 - accuracy: 0. - ETA: - ETA: 16s - loss: 1.6028 - accuracy:  - ETA: 16s - loss: 1.6023 - accuracy: - ETA: 15s - loss: 1. - ETA: 14s - loss: 1.5983 - accuracy: - ETA: 13s - loss: 1.59 - ETA: 12s - loss: 1.5963 - accuracy: 0.354 - ETA: 12s - loss: 1.5962 - - ETA: 11s - loss: 1.5958 - accuracy: - ETA: 11s - loss: 1.5962 - ac - ETA: 10s - loss: 1.5956 - accu - ETA: 8s - loss: - ETA: 6s - loss: 1.5937 - accuracy: 0. - ETA: 6s - l - E - ETA - ETA: 1s - loss: 1.5911 -  - ETA
Epoch 17/30
36000/36000 [==============================] - 34s

KeyboardInterrupt: 

In [ ]:
X_shuffled, y_shuffled = shuffle_data(X, y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_shuffled, y_shuffled, test_size=0.3)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
#X_mean = np.mean(X_train, axis=0)

In [ ]:
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)

In [ ]:
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
input_shape = X_train[0].shape

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs = 25

hist = model.fit(X_train, y_train, 
          batch_size=batch_size,
          epochs=epochs, 
          verbose=1,
          validation_data=(X_valid, y_valid))

In [ ]:
score = model.evaluate(X_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save('model.h5')